In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import os

print("--- Part 2: Creating the Initial LSTM Model ---")

# --- Configuration ---
INPUT_DIR = r"D:\FedShield,Personal\wataiData\csv\CICIoT2023\federated_data"
SERVER_DF_PATH = os.path.join(INPUT_DIR, 'server_df.csv')

# --- Load Server Data ---
print(f"Loading server data from {SERVER_DF_PATH}...")
server_df = pd.read_csv(SERVER_DF_PATH)

# --- Preprocessing ---
print("Preprocessing data...")
X_server = server_df.drop('label', axis=1)
y_server = server_df['label']

label_encoder = LabelEncoder()
y_server_encoded = label_encoder.fit_transform(y_server)

scaler = StandardScaler()
X_server_scaled = scaler.fit_transform(X_server)

# --- Save the preprocessors ---
with open(os.path.join(INPUT_DIR, 'label_encoder.pkl'), 'wb') as f:
    pickle.dump(label_encoder, f)
with open(os.path.join(INPUT_DIR, 'scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)
print("Saved label_encoder.pkl and scaler.pkl.")

# --- Reshape Data for LSTM ---
# The LSTM layer expects a 3D input: (samples, timesteps, features)
# We'll treat each row as 1 timestep with N features.
n_features = X_server_scaled.shape[1]
X_server_reshaped = X_server_scaled.reshape(-1, 1, n_features)

print(f"Original data shape: {X_server_scaled.shape}")
print(f"Reshaped data for LSTM: {X_server_reshaped.shape}")

# --- Build and Train LSTM Model ---
print("Building and training the LSTM model...")

def create_lstm_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        # LSTM layer to process the sequence
        tf.keras.layers.LSTM(64, input_shape=input_shape),
        
        # Standard Dense layer to make the final classification
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# The input shape is now (timesteps, features)
lstm_input_shape = (X_server_reshaped.shape[1], X_server_reshaped.shape[2])
initial_model = create_lstm_model(lstm_input_shape, len(label_encoder.classes_))

initial_model.summary()

initial_model.fit(X_server_reshaped, y_server_encoded, epochs=5, batch_size=256, verbose=1)

# --- Extract and Save Weights ---
print("Extracting and saving model weights...")
initial_weights = initial_model.get_weights()
with open(os.path.join(INPUT_DIR, 'initial_weights_lstm.pkl'), 'wb') as f:
    pickle.dump(initial_weights, f)

print(f"\n✅ Part 2 (LSTM) Complete!")
print(f"Initial LSTM model weights saved to: {os.path.join(INPUT_DIR, 'initial_weights_lstm.pkl')}")

--- Part 2: Creating the Initial LSTM Model ---
Loading server data from D:\FedShield,Personal\wataiData\csv\CICIoT2023\federated_data\server_df.csv...
Preprocessing data...
Saved label_encoder.pkl and scaler.pkl.
Original data shape: (3689999, 46)
Reshaped data for LSTM: (3689999, 1, 46)
Building and training the LSTM model...


d:\FedShield,Personal\CICIoT2023\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        28,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 34)             │         1,122 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,618 (123.51 KB)

 Trainable params: 31,618 (123.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
14415/14415 ━━━━━━━━━━━━━━━━━━━━ 43s 3ms/step - accuracy: 0.9045 - loss: 0.2586
Epoch 2/5
14415/14415 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9839 - loss: 0.0544
Epoch 3/5
14415/14415 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9851 - loss: 0.0465
Epoch 4/5
14415/14415 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9859 - loss: 0.0433
Epoch 5/5
14415/14415 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9862 - loss: 0.0418
Extracting and saving model weights...

✅ Part 2 (LSTM) Complete!
Initial LSTM model weights saved to: D:\FedShield,Personal\wataiData\csv\CICIoT2023\federated_data\initial_weights_lstm.pkl
